In [1]:
import pandas as pd
import numpy as np

In [2]:
import pyarrow

In [3]:
file_path = 'yellow_tripdata_2023-01.parquet'

df = pd.read_parquet(file_path)

print(df.head())

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1         2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2         2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3         1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4         2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           0.97         1.0                  N           161           141   
1           1.10         1.0                  N            43           237   
2           2.51         1.0                  N            48           238   
3           1.90         1.0                  N           138             7   
4           1.43         1.0                  N           107            79   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
print(df.describe())

           VendorID        tpep_pickup_datetime       tpep_dropoff_datetime  \
count  3.066766e+06                     3066766                     3066766   
mean   1.730215e+00  2023-01-17 00:22:26.288164  2023-01-17 00:38:06.427874   
min    1.000000e+00         2008-12-31 23:01:42         2009-01-01 14:29:11   
25%    1.000000e+00  2023-01-09 16:21:57.250000         2023-01-09 16:37:06   
50%    2.000000e+00  2023-01-17 08:42:29.500000  2023-01-17 08:58:30.500000   
75%    2.000000e+00         2023-01-24 16:26:27         2023-01-24 16:42:49   
max    2.000000e+00         2023-02-01 00:56:53         2023-02-02 09:28:47   
std    4.438480e-01                         NaN                         NaN   

       passenger_count  trip_distance    RatecodeID  PULocationID  \
count     2.995023e+06   3.066766e+06  2.995023e+06  3.066766e+06   
mean      1.362532e+00   3.847342e+00  1.497440e+00  1.663980e+02   
min       0.000000e+00   0.000000e+00  1.000000e+00  1.000000e+00   
25%       1.

In [8]:
#Q1

num_columns = df.shape[1]
num_columns

19

In [15]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [23]:
df['duration_minutes'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [24]:
january_trips = df[df['tpep_pickup_datetime'].dt.month == 1]

std_duration = january_trips['duration_minutes'].std()

In [25]:
#Q2

std_duration

42.585641764259044

In [26]:
# Filter records to keep only those with duration between 1 and 60 minutes
filtered_trips = df[(df['duration_minutes'] >= 1) & (df['duration_minutes'] <= 60)]

# Calculate the fraction of records left after dropping the outliers
fraction_remaining = len(filtered_trips) / len(df)

In [28]:
#Q3

fraction_remaining

0.9812202822125979

In [31]:
from sklearn.feature_extraction import DictVectorizer

In [33]:
subset_df = df[['PULocationID', 'DOLocationID']]

In [37]:
#Q4

subset_df = df[['PULocationID', 'DOLocationID']].copy()

subset_df.loc[:, 'PULocationID'] = subset_df['PULocationID'].astype(str)
subset_df.loc[:, 'DOLocationID'] = subset_df['DOLocationID'].astype(str)

data_dicts = subset_df.to_dict(orient='records')

vec = DictVectorizer(sparse=False)
feature_matrix = vec.fit_transform(data_dicts)

feature_names = vec.get_feature_names_out()

num_columns = feature_matrix.shape[1]

print(f"The dimensionality of the matrix is: {num_columns} columns")

The dimensionality of the matrix is: 518 columns


In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
y = df['duration_minutes']

# Train a linear regression model
model = LinearRegression()
model.fit(feature_matrix, y)

# Make predictions on the training data
predictions = model.predict(feature_matrix)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y, predictions))

In [ ]:
#6

file_path = 'yellow_tripdata_2023-02.parquet'

df_feb = pd.read_parquet(file_path)

df_feb['duration_minutes'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds() / 60

subset_df_feb = df_feb[['PULocationID', 'DOLocationID']]

# Convert location IDs to strings using .loc to avoid the SettingWithCopyWarning
subset_df_feb.loc[:, 'PULocationID'] = subset_df_feb['PULocationID'].astype(str)
subset_df_feb.loc[:, 'DOLocationID'] = subset_df_feb['DOLocationID'].astype(str)

# Convert DataFrame to a list of dictionaries
val_dicts = subset_df_feb.to_dict(orient='records')

# Transform the validation data using the same vectorizer
X_val = vec.transform(val_dicts)

# Prepare the target variable for validation
y_val = df_feb['duration_minutes']

# Make predictions on the validation data
predictions = model.predict(X_val)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_val, predictions))